In [81]:
import pandas as pd
import glob
import os
import pathlib
import re
import unicodedata

In [82]:
#FUNCTIONS

#=====parsing strings for height calculations=====
def height_parser(str_in):
    height = re.findall(r'(\d{1,3}(?:\.\d+)?)', str_in)
    h_num = [float(i) for i in height]
    conversion_unit = 12 #inches
    inches = (h_num[0]*conversion_unit) + h_num[1]
    return inches

#=====standardize names=====
def clean_names_dropna(df_in, col='name'):
    df_in = df_in.dropna(axis='index', how='any').copy()
    df_in[col] = (df_in[col]
    .str.normalize('NFKD')
    .str.encode('ascii','ignore')
    .str.decode('utf-8') 
    .str.replace("'","") #e.g., o'donnel
    .str.replace("  "," ") #just in case
    .str.strip()
    .str.lower()).copy()
    return df_in

#=====create new columns=====
def proc_aio(df_in):
    columns_raw = ['height','wingspan','relative']
    columns_new = ['height_inches', 'wingspan_inches', 'relative']
    df_in = clean_names_dropna(df_in).copy()
    for old, new in zip(columns_raw[:2], columns_new[:2]):
        if new not in df_in.columns:
            df_in[new] = df_in[old].apply(height_parser)
            print(f'created {new} from {old}\n')
    if columns_new[2] not in df_in.columns:
        df_in[columns_new[2]] = df_in[columns_new[1]] - df_in[columns_new[0]]
        print(f'created {columns_new[2]}')

    df_in = df_in[['name','height_inches', 'wingspan_inches', 'relative']]
    return df_in

In [83]:
#======= CLEANING NBAORG DATA ========

In [84]:
subdirectory = '00-raw/Wingspan_Raw/wingspan_nbaorg'
target_directory = pathlib.Path.cwd() / subdirectory
file_list = list(target_directory.glob('*.csv'))

print(f'datasets merged: {len(file_list)}')

df = pd.DataFrame()

for file in file_list: #combining raw CSVs
    add_df = pd.read_csv(file)
    df = pd.concat([df, add_df], ignore_index = True)

print(df.shape)

datasets merged: 20
(1313, 10)


In [85]:
#rename columns: need name, height, wingspan - and for later, way to 
names = {
    'Crom_text__NpR1_':'name',
    'tablescraper-selected-row 4':'height',
    'tablescraper-selected-row 8':'wingspan'
}

df = df.rename(columns=names)
df = df[['name','height','wingspan']]
wingspan_nbaorg_combined = proc_aio(df)

wingspan_nbaorg_combined.to_csv('01-interim/wingspan_cleaned/wingspan_nbaorg_combined.csv', index=False)
wingspan_nbaorg_combined.head(1)

created height_inches from height

created wingspan_inches from wingspan

created relative


,name,height_inches,wingspan_inches,relative
0,kenny adeleke,78.5,82.25,3.75


In [86]:
#========= CLEANING 2025 WINGSPAN =========

In [87]:
file_name = 'wingspan_crafted_2025'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')
print(f'raw column names: {df.columns}\n\n')
df = df[['name','height_inches','wingspan_inches','wingspan_advantage']]
df.columns = ['name','height_inches','wingspan_inches','relative']
wingspan_crafted_2025 = proc_aio(df)

wingspan_crafted_2025.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv', index=False)
wingspan_crafted_2025.head(1)

raw column names: Index(['name', 'team', 'position', 'height_inches', 'wingspan_inches',
       'wingspan_advantage', 'g', 'mp', 'pts', 'orb', 'drb', 'trb', 'ast',
       'stl', 'blk', 'o_rtg', 'd_rtg', 'e_fg_percent', 'ft_percent',
       'fg_percent', 'avg_dist_of_fg', '2pt_rate', '3pt_rate', '2pt_percent',
       '3pt_percent', 'dunk_rate'],
      dtype='object')




,name,height_inches,wingspan_inches,relative
0,aj johnson,76.25,80.5,4.25


In [88]:
#========= CLEANING 2026 WINGSPAN =========
file_name = 'wingspan_crafted_2026'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')

names = {
    'the-name':'name',
    'tablescraper-selected-row':'height',
    'tablescraper-selected-row 2':'wingspan',
    'tablescraper-selected-row 3':'relative'
}
df = df.rename(columns=names)
df = df[['name','height','wingspan','relative']]
df = proc_aio(df)

wingspan_crafted_2026 = df

wingspan_crafted_2026.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv', index=False)
wingspan_crafted_2026.head(1)

created height_inches from height

created wingspan_inches from wingspan



,name,height_inches,wingspan_inches,relative
0,mo bamba,83.25,94.0,10.75


In [92]:
#========= WINGSPAN_KAGGLE =========
file_name = 'wingspan_Kaggle'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')
names = {
    'PLAYER':'name',
    'HGT':'height_inches',
    'WNGSPN':'wingspan_inches'
}

df = df.rename(columns=names)
df = df[['name','height_inches','wingspan_inches']]

df['name'] = df['name'].str.split(',').apply(
    lambda x: f"{x[1].strip()} {x[0].strip()}" if len(x) > 1 else x[0]
)
df.head(1)
wingspan_Kaggle = proc_aio(df)

wingspan_Kaggle.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv', index=False)
wingspan_Kaggle.head(1)

created relative


,name,height_inches,wingspan_inches,relative
0,izan almansa,81.25,85.75,4.5


In [90]:
#=========== FINISHED INDIVIDUAL PROC, NOW MERGE ==============

In [91]:
list_source = [wingspan_nbaorg_combined,
               wingspan_crafted_2025,
               wingspan_crafted_2026,
               wingspan_Kaggle]


df_merge = pd.DataFrame()

for f in list_source:
    df_merge = pd.concat([df_merge,f],axis='index')
    
print(df_merge.shape)
df_merge = df_merge.drop_duplicates()
df_merge['relative'] = df_merge['relative'].round(2)
print(df_merge.shape)

df_merge.to_csv(f'01-interim/wingspan_cleaned/combined_cleaned.csv', index=False)

(3669, 4)
(1911, 4)
